In [26]:
import subprocess
import sys
import re
import itertools
import pandas as pd

In [47]:
def read_db(db_file, allowed_ids=None):
    """
    Lê o ficheiro de base de dados e retorna um dicionário com as sequências cujos IDs estão em allowed_ids.
    Se allowed_ids for None, retorna todas as sequências.
    """
    sequences = {}
    current_id = None
    current_seq = ""
    with open(db_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.rstrip()
            if not line:
                continue
            if line.startswith('@'):
                # Guarda a sequência anterior, se o ID for permitido
                if current_id is not None and current_seq:
                    if allowed_ids is None or current_id in allowed_ids:
                        sequences[current_id] = current_seq
                current_id = line[1:].strip()
                current_seq = ""
            else:
                current_seq += line.strip()

        # Última sequência
        if current_id is not None and current_seq:
            if allowed_ids is None or current_id in allowed_ids:
                sequences[current_id] = current_seq

    return sequences

def run_levenshtein_similarity(executable, db_file, id1, id2):
    """
    Invoca o programa C++ com os parâmetros -db, -id1 e -id2 e retorna o valor da similaridade.
    Se ocorrer algum erro ou o valor não for extraído, retorna None.
    """
    try:
        # Chamada do programa (certifique-se de que o caminho do executável está correto)
        result = subprocess.run([executable, "-db", db_file, "-id1", id1, "-id2", id2],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    except Exception as e:
        print(f"Erro ao executar o programa: {e}")
        return None

    # Procura pela string "Similaridade: " no output
    match = re.search(r"Similaridade:\s*([\d\.]+)", result.stdout)
    if match:
        try:
            sim = float(match.group(1))
            return sim
        except ValueError:
            print(f"Erro ao converter a similaridade extraída: {match.group(1)}")
            return None
    else:
        # Se não encontrar, mostra a saída para ajudar no debug
        print(f"Não foi possível extrair similaridade para os IDs:\n  id1: {id1}\n  id2: {id2}")
        print("Saída do programa:", result.stdout)
        print("Erro:", result.stderr)
        return None

def run_MRC_similarity(executable, db_file, id1, id2, a, k):
    """
    Invoca o programa C++ com os parâmetros -db, -id1 e -id2 e retorna o valor da similaridade.
    Se ocorrer algum erro ou o valor não for extraído, retorna None.
    """
    try:
        # Chamada do programa (certifique-se de que o caminho do executável está correto)
        result = subprocess.run([executable, "-db", db_file, "-id1", id1, "-id2", id2, "-a", a, "-k", k],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    except Exception as e:
        print(f"Erro ao executar o programa: {e}")
        return None

    match = re.search(r"NRC médio:\s*([\d\.]+)", result.stdout)
    if match:
        try:
            sim = float(match.group(1))
            return sim
        except ValueError:
            print(f"Erro ao converter a similaridade extraída: {match.group(1)}")
            return None
    else:
        # Se não encontrar, mostra a saída para ajudar no debug
        print(f"Não foi possível extrair similaridade para os IDs:\n  id1: {id1}\n  id2: {id2}")
        print("Saída do programa:", result.stdout)
        print("Erro:", result.stderr)
        return None
    
def read_main_output(executable, db_file, model_file, a, top):
    try:
        result = subprocess.run([
            executable,
            "-db", db_file,
            "-m", model_file,
            "-a", str(a),
            "-t", str(top)
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        output = result.stdout

        # Expressão regular que extrai (ID, NRC)
        matches = re.findall(r'\d+\.\s+(.*?)\s+- NRC:\s+([\d\.]+)', output)

        # Filtra apenas os que têm NRC < 1
        filtered_ids = [seq_id.strip() for seq_id, nrc in matches if float(nrc) < 1.0]

        return filtered_ids

    except Exception as e:
        print(f"Erro ao executar o programa: {e}")
        return []

In [46]:
def main():
    db_file = "../txt_files/db_5.txt"
    similarity_executable = "../src/bin/similarities_models.out"
    a = "0.001"
    k = "11"

    # Obtém os IDs com NRC < 1
    sequence_ids = ["New", "NewMutation1", "NewMutation2", "NewMutation5", "NewMutation10", "NewMutation20", "NewMutation50"]

    # Lê apenas as sequências desses IDs
    sequences = read_db(db_file, allowed_ids=set(sequence_ids))
    if not sequences:
        print("Nenhuma sequência correspondente foi encontrada na base de dados.")
        sys.exit(1)

    ids = list(sequences.keys())

    results = []

    # Compara todas as combinações possíveis entre essas sequências
    for id1, id2 in itertools.combinations(ids, 2):
        sim = run_MRC_similarity(similarity_executable, db_file, id1, id2, a, k)
        if sim is not None and id1 == "New":
            results.append({
                'ID1': id1,
                'ID2': id2,
                'NRC': sim
            })

    # Cria e exibe a tabela com os resultados
    df = pd.DataFrame(results)
    
    if df.empty:
        print("Nenhuma similaridade foi calculada.")
    else:
        # Mostrar a tabela
        print("\nTabela de Similaridades:")
        print(df.to_string(index=False))

if __name__ == '__main__':
    main()


Tabela de Similaridades:
ID1           ID2      NRC
New  NewMutation1 0.126870
New  NewMutation2 0.281762
New  NewMutation5 0.555225
New NewMutation10 0.859063
New NewMutation20 1.006150
New NewMutation50 1.007170


In [ ]:
def main():
    db_file = "../txt_files/db_5.txt"
    similarity_executable = "../src/bin/similarities_levenshtein.out"

    # Obtém os IDs com NRC < 1
    sequence_ids = ["New", "NewMutation1", "NewMutation2", "NewMutation5", "NewMutation10", "NewMutation20", "NewMutation50"]

    # Lê apenas as sequências desses IDs
    sequences = read_db(db_file, allowed_ids=set(sequence_ids))
    if not sequences:
        print("Nenhuma sequência correspondente foi encontrada na base de dados.")
        sys.exit(1)

    ids = list(sequences.keys())

    results = []

    # Compara todas as combinações possíveis entre essas sequências
    for id1, id2 in itertools.combinations(ids, 2):
        sim = run_levenshtein_similarity(similarity_executable, db_file, id1, id2)
        if sim is not None and id1 == "New":
            results.append({
                'ID1': id1,
                'ID2': id2,
                'Similaridade': sim
            })

    # Cria e exibe a tabela com os resultados
    df = pd.DataFrame(results)
    
    if df.empty:
        print("Nenhuma similaridade foi calculada.")
    else:
        # Opcional: filtrar os pares com similaridade < 1.0
        df = df[df['Similaridade'] < 1.0].reset_index(drop=True)

        # Mostrar a tabela
        print("\nTabela de NRC:")
        print(df.to_string(index=False))

if __name__ == '__main__':
    main()


Tabela de Similaridades:
ID1           ID2  Similaridade
New  NewMutation1        0.9918
New  NewMutation2        0.9802
New  NewMutation5        0.9536
New NewMutation10        0.9015
New NewMutation20        0.8065
New NewMutation50        0.5450


In [30]:
def main():
    db_file = "../txt_files/db.txt"
    model_file = "../models/k11.bin"
    a = "0.001"
    top = "20"
    main_executable = "../src/bin/main.out"
    similarity_executable = "../src/bin/similarities_levenshtein.out"

    # Obtém os IDs com NRC < 1
    sequence_ids = read_main_output(main_executable, db_file, model_file, a, top)
    if not sequence_ids:
        print("Nenhum ID com NRC < 1 foi encontrado.")
        return

    # Lê apenas as sequências desses IDs
    sequences = read_db(db_file, allowed_ids=set(sequence_ids))
    if not sequences:
        print("Nenhuma sequência correspondente foi encontrada na base de dados.")
        sys.exit(1)

    ids = list(sequences.keys())

    results = []

    # Compara todas as combinações possíveis entre essas sequências
    for id1, id2 in itertools.combinations(ids, 2):
        sim = run_levenshtein_similarity(similarity_executable, db_file, id1, id2)
        if sim is not None:
            results.append({
                'ID1': id1,
                'ID2': id2,
                'Similaridade': sim
            })

    # Cria e exibe a tabela com os resultados
    df = pd.DataFrame(results)
    
    if df.empty:
        print("Nenhuma similaridade foi calculada.")
    else:
        # Opcional: filtrar os pares com similaridade < 1.0
        df = df[df['Similaridade'] < 1.0].reset_index(drop=True)

        # Mostrar a tabela
        print("\nTabela de Similaridades:")
        print(df.to_string(index=False))

if __name__ == '__main__':
    main()


Tabela de Similaridades:
                                                                 ID1                                                                  ID2  Similaridade
                                                    Super ISS Si1240 gi|49169782|ref|NC_005831.2| Human Coronavirus NL63, complete genome      0.045004
                                                    Super ISS Si1240                     NC_001348.1 Human herpesvirus 3, complete genome      0.009929
                                                    Super ISS Si1240                  NC_005831.2 Human Coronavirus NL63, complete genome      0.045004
                                                    Super ISS Si1240                                                        Super MUL 720      0.451613
                                                    Super ISS Si1240           OR353425.1 Octopus vulgaris mitochondrion, complete genome      0.079193
gi|49169782|ref|NC_005831.2| Human Coronavirus NL63, complete 

In [48]:
def main():
    db_file = "../txt_files/db.txt"
    model_file = "../models/k11.bin"
    a = "0.001"
    k = "11"
    top = "20"
    main_executable = "../src/bin/main.out"
    similarity_executable = "../src/bin/similarities_models.out"

    # Obtém os IDs com NRC < 1
    sequence_ids = read_main_output(main_executable, db_file, model_file, a, top)
    if not sequence_ids:
        print("Nenhum ID com NRC < 1 foi encontrado.")
        return

    # Lê apenas as sequências desses IDs
    sequences = read_db(db_file, allowed_ids=set(sequence_ids))
    if not sequences:
        print("Nenhuma sequência correspondente foi encontrada na base de dados.")
        sys.exit(1)

    ids = list(sequences.keys())

    results = []

    # Compara todas as combinações possíveis entre essas sequências
    for id1, id2 in itertools.combinations(ids, 2):
        sim = run_MRC_similarity(similarity_executable, db_file, id1, id2, a, k)
        if sim is not None:
            results.append({
                'ID1': id1,
                'ID2': id2,
                'NRC': sim
            })

    # Cria e exibe a tabela com os resultados
    df = pd.DataFrame(results)
    
    if df.empty:
        print("Nenhuma similaridade foi calculada.")
    else:
        # Mostrar a tabela
        print("\nTabela de NRC:")
        print(df.to_string(index=False))

if __name__ == '__main__':
    main()


Tabela de NRC:
                                                                 ID1                                                                  ID2      NRC
                                                    Super ISS Si1240 gi|49169782|ref|NC_005831.2| Human Coronavirus NL63, complete genome 1.014690
                                                    Super ISS Si1240                     NC_001348.1 Human herpesvirus 3, complete genome 1.040380
                                                    Super ISS Si1240                  NC_005831.2 Human Coronavirus NL63, complete genome 1.016790
                                                    Super ISS Si1240                                                        Super MUL 720 1.003280
                                                    Super ISS Si1240           OR353425.1 Octopus vulgaris mitochondrion, complete genome 1.006940
gi|49169782|ref|NC_005831.2| Human Coronavirus NL63, complete genome                     NC_001348.1 H